In [ ]:
# Continuation of House Prices from HW1

In [25]:
# Import
import pandas as pd
import torch

train_40 = pd.read_csv("./data-from-hw1/train_40.csv")
test_40 = pd.read_csv("./data-from-hw1/test_40.csv")
prices = pd.read_csv('./data-from-hw1/train.csv')['SalePrice']
print(train_40.shape)

(1460, 40)


In [ ]:
# Split training set into training and validation


In [53]:
# Define model class as linear regression
# y = xW + b
W = torch.randn(40, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
train_40_tensor = torch.tensor(train_40.values, dtype = torch.float32)

In [57]:
# Define one autodiff/update loop
def update_once(learning_rate, W, b):
    prices_predicted = train_40_tensor @ W + b
    loss = torch.mean((prices_predicted - torch.tensor(prices))**2)
    loss.backward()
    with torch.no_grad():  # disables autograd temporarily
        W.data -= learning_rate * W.grad
        b.data -= learning_rate * b.grad

    W.grad.zero_()
    b.grad.zero_()
    return loss, W, b

In [58]:
# Define how many updates
def update_N(learning_rate, num_epochs, W, b):
    for i in range(num_epochs):
        loss, W, b = update_once(learning_rate, W, b)
        print(f"Epoch {i+1}, Loss: {loss}")
    return W, b

In [59]:
# Try some hyperparameters
update_N(0.01, 200, W, b)

Epoch 1, Loss: 39038218240.0
Epoch 2, Loss: 16378866688.0
Epoch 3, Loss: 13554603008.0
Epoch 4, Loss: 11560742912.0
Epoch 5, Loss: 10152348672.0
Epoch 6, Loss: 9156776960.0
Epoch 7, Loss: 8452341760.0
Epoch 8, Loss: 7953268224.0
Epoch 9, Loss: 7599089152.0
Epoch 10, Loss: 7347180544.0
Epoch 11, Loss: 7167493120.0
Epoch 12, Loss: 7038835200.0
Epoch 13, Loss: 6946266112.0
Epoch 14, Loss: 6879245824.0
Epoch 15, Loss: 6830339584.0
Epoch 16, Loss: 6794299392.0
Epoch 17, Loss: 6767419904.0
Epoch 18, Loss: 6747084288.0
Epoch 19, Loss: 6731442688.0
Epoch 20, Loss: 6719184896.0
Epoch 21, Loss: 6709383168.0
Epoch 22, Loss: 6701378560.0
Epoch 23, Loss: 6694702080.0
Epoch 24, Loss: 6689019904.0
Epoch 25, Loss: 6684092928.0
Epoch 26, Loss: 6679748096.0
Epoch 27, Loss: 6675860992.0
Epoch 28, Loss: 6672339968.0
Epoch 29, Loss: 6669118464.0
Epoch 30, Loss: 6666146304.0
Epoch 31, Loss: 6663386112.0
Epoch 32, Loss: 6660809216.0
Epoch 33, Loss: 6658391552.0
Epoch 34, Loss: 6656116736.0
Epoch 35, Loss: 66

(tensor([[14259.0586],
         [ 6830.2446],
         [ 5577.6338],
         [16200.3252],
         [ 4008.0037],
         [ 5604.4062],
         [ 7660.2124],
         [ 4713.6489],
         [ 6479.4224],
         [19096.9043],
         [18769.4043],
         [12985.6768],
         [ 9912.0811],
         [14805.9072],
         [ 9838.2217],
         [ 7800.0835],
         [ 4132.2969],
         [ 7924.4771],
         [ 7836.9702],
         [12565.3174],
         [11769.3779],
         [ 6739.9434],
         [ 1136.2297],
         [ 5060.3770],
         [ 1489.3124],
         [ 7912.7773],
         [ 1966.0243],
         [ 5961.5674],
         [ 7256.7959],
         [ 7744.2925],
         [ 6433.7222],
         [  932.3957],
         [16230.9883],
         [ 2674.9314],
         [ 3837.1965],
         [ 8006.7866],
         [ 2200.6814],
         [ 9900.4609],
         [ 7717.9512],
         [ 9055.6260]], requires_grad=True),
 tensor([30082.8438], requires_grad=True))